Microservices - ESI
-------------------

![](images/Microservices-ESI.png)

Quelle: Buch Microservices Rezepte
- - -


Das [ESI Beispiel](https://github.com/mc-b/scs-esi) zeigt mittels ESI-Integration, wie Fragmente eines Common-Microservices in andere Microservices integriert werden können.

Das Beispiel besteht aus
* **scs-demo-esi-common** stellt allgemeine Kopf- und Fusszeilen für Services bereit. Es ist in Go geschrieben.
* **scs-demo-esi-order** erledigt die Auftragsbearbeitung. Es verwendet Spring-Boot und ist in Java geschrieben.
* **Varnish** interpretiert die ESI-Befehle (Edge Side Includes)

**ESI**

[Edge Side Includes](https://de.wikipedia.org/wiki/Edge_Side_Includes) oder ESI genannt, ist eine Auszeichnungssprache um Webseiten-Komponenten dynamisch zusammenzubauen und diese als fertige Seite auszuliefern. 



- - -
Als erstes starten wir alle drei Microservices und Schauen uns das Resultat mittels öffnen des angezeiten URLs an:

In [11]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/SCS-ESI.yaml
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service varnish --namespace scsesi -o=jsonpath='{ .spec.ports[0].nodePort }')/"

service/common unchanged
deployment.apps/common unchanged
service/order unchanged
deployment.apps/order unchanged
service/varnish unchanged
deployment.apps/scsesi-varnish unchanged
http://192.168.137.100:32080

- - -

Um zu Verstehen was genau passiert müssen wir uns den produzierten HTML Code der Fachapplikation **scs-demo-esi-order** anschauen.

Deshalb starten wie diesen Microservices separat und schauen uns, im Browser, mittels `Seitenquelltext anzeigen` den produzierten HTML Code an


In [14]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/SCS-ESI-order.yaml
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service order-standalone --namespace scsesi -o=jsonpath='{ .spec.ports[0].nodePort }')/"

service/order-standalone unchanged
deployment.apps/order-standalone unchanged
http://192.168.137.100:32090

Wie die Microservices zusammenspielen, lässt sich visuell in der Weave Scope Oberfläche verfolgen.

Mittels den nachfolgenden Befehlen wird die Weave Scope Oberfläche freigeschaltet.

In [23]:
! kubectl get service weave-scope-app -n weave -o yaml | sed 's/ClusterIP/NodePort/' | kubectl apply -f -
! echo $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service weave-scope-app --namespace weave -o=jsonpath='{ .spec.ports[0].nodePort }')/")'/#!/state/{"topologyId":"containers","topologyOptions":{"containers":{"namespace":["scsesi"]},"pods":{"namespace":["scsesi"]}}}'

service/weave-scope-app configured
http://192.168.137.100:30230/#!/state/{"topologyId":"containers","topologyOptions":{"containers":{"namespace":["scsesi"]},"pods":{"namespace":["scsesi"]}}}


Aufräumen

In [24]:
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/SCS-ESI-order.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/SCS-ESI.yaml

service "order-standalone" deleted
deployment.apps "order-standalone" deleted
service "common" deleted
deployment.apps "common" deleted
service "order" deleted
deployment.apps "order" deleted
service "varnish" deleted
deployment.apps "scsesi-varnish" deleted
